---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
pd.set_option('display.max_rows',300)

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    ut=pd.read_fwf('university_towns.txt')
    states=['Alabama']
    reg=[]
    d=pd.DataFrame({'Alabama[edit]':'Alabama'},index=[0])
    ut=pd.concat([d,ut]).reset_index(drop=True)
    ut.columns=['name']

    for index,row in ut.iterrows():
        data=row['name']
        new_data=''.join([i for i in data if not i.isdigit()])
        i=new_data.find('[')
        j=new_data.find('(')
        if i>j and j==-1:
            states.append(new_data[:i].strip())
        else:
            reg.append(new_data[:j].strip())

    def chg(data):
        new_data=''.join([i for i in data if not i.isdigit()])
        i=new_data.find('[')
        j=new_data.find('(')
        if i>j and j==-1:
            return new_data[:i].strip()
        elif i==j:
            return new_data.strip()
        elif data=='Alabama':
            return data
        else:
            return new_data[:j].strip()
        
    ut['name']=ut['name'].apply(chg)
    lst=[]
    j=0
    ut.head(20)

    for index,row in ut.iterrows():
            if j<len(states) and  row['name']==states[j]:
                lst.append(states[j])
                if j<=len(states)-1:
                    j=j+1       
            elif j<=len(states):
                lst.append(states[j-1])

    ut['name'].head(20)
    ut['st']=lst
    ut.head(20)

    ut=ut[ut.name!=ut['st']]
    col_name='st'
    fc=ut.pop(col_name)
    ut.insert(0,col_name,fc)
    ut.columns=["State", "RegionName"]
    ut.reset_index(drop=True,inplace=True)
    
    return ut


In [60]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    rs=pd.read_excel('gdplev.xls',skiprows=7)[212:]
    rs =rs[['Unnamed: 4','Unnamed: 5']]
    rs=rs.reset_index(drop=True)
    for index,row in rs.iterrows():
        if rs.iloc[index+2][1]<rs.iloc[index+1][1] and rs.iloc[index+1][1] <rs.iloc[index][1]:
            return rs.iloc[index][0]

In [78]:
def get_recession_end ():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    rs=pd.read_excel('gdplev.xls',skiprows=7)[212:]
    rs =rs[['Unnamed: 4','Unnamed: 5']]
    rs=rs.reset_index(drop=True)
    for index,row in rs.iterrows():
        if rs.iloc[index+2][1]<rs.iloc[index+1][1] and rs.iloc[index+1][1] <rs.iloc[index][1]:
            a=index
            break
    rs=rs[a:].reset_index(drop=True)
    for index,row in rs.iterrows():
        if rs.iloc[index][1]<rs.iloc[index+1][1] and rs.iloc[index+1][1]<rs.iloc[index+2][1]:
            return rs.iloc[index+2][0]


'2009q4'

In [80]:
def get_recession_bottom():
    rs=pd.read_excel('gdplev.xls',skiprows=7)[212:]
    rs =rs[['Unnamed: 4','Unnamed: 5']]
    rs=rs.reset_index(drop=True)
    for index,row in rs.iterrows():
        if rs.iloc[index+2][1]<rs.iloc[index+1][1] and rs.iloc[index+1][1] <rs.iloc[index][1]:
            a=index
            break
    rs=rs[a:].reset_index(drop=True)
    for index,row in rs.iterrows():
        if rs.iloc[index][1]<rs.iloc[index+1][1]:
            return rs.iloc[index][0]

'2009q2'

In [138]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    h=pd.read_csv('City_Zhvi_AllHomes.csv')
    l=list(h.columns)
    l=[i for i in l if i[0]!='1']
    h=h[l]
    h=h.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)
    h=h.set_index(['State','RegionName'])
    h=h.groupby(np.arange(len(h.columns))//3,axis=1).mean()
    s=[]
    for i in range(2000,2017):
        s.append(str(i)+'q1')
        s.append(str(i)+'q2')
        s.append(str(i)+'q3')
        s.append(str(i)+'q4')
    h.columns=s[:-1]
    h=h.rename(index=states)
    return h

In [193]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    utown=get_list_of_university_towns()
    rstart=get_recession_start()
    rbottom=get_recession_bottom()
    h=convert_housing_data_to_quarters()
    rbefore=h.columns[h.columns.get_loc(rstart)-1]
    h['Priceratio']=h[rstart]-h[rbefore]
    item_to_select=list(utown[['State']])
    h=h[[rstart,rbottom,'Priceratio']].reset_index()
    g1=pd.merge(utown,h,how='inner',on=['State','RegionName'])
    g2=pd.merge(h,g1,how='outer',on=['State','RegionName',rstart,rbottom,'Priceratio'])
    s,p=ttest_ind(g1['Priceratio'].dropna(),g2['Priceratio'].dropna())
    different=True if p<0.01 else False
    better="non-university town" if g1['Priceratio'].mean()<g2['Priceratio'].mean() else 'university town'
    return different,p,better

(True, 0.0031302524022853353, 'university town')